# Gelman Rubin Convergence Test for MCMC Algorithm

In [7]:
import numpy as np
import matplotlib.pyplot as plt
def eta(a,omega):
    s=((1-omega)/omega)**(1/3)
    return 2*((s**3+1)**(0.5))*((1/(a**4))-0.1540*(s/(a**3))+0.4304*(s**2)/(a**2)+
                                0.19097*((s**3)/a)+0.066941*(s**4))**(-1/8)
def Dls(z,omega):
    return 3000*(1+z)*(eta(1,omega)-eta(1/(1+z),omega))
    
def mu_th(z,omega,h):
    return 25-5*np.log10(h)+5*np.log10(Dls(z,omega))
z_ob,mu_ob=np.loadtxt(r"./jla_mub.txt",
               delimiter=' ',unpack=True)
Cov1=np.loadtxt(r"./jla_mub_covmatrix.txt")
Cov=np.reshape(Cov1,(31,31))
Incov=np.linalg.inv(Cov)

In [8]:
# generating random number for omega_m and h some where between 0 to 1
omega=np.random.uniform()
h=np.random.uniform()
n=500
D=np.empty(31)
chain=25 

def lnL(omega,h):   
    if(omega<=0 or h<=0):
        logL = -1.e100 
    else:
        for j in range (31):
            D[j] = mu_ob[j]-mu_th(z_ob[j],omega,h)
        logL = -0.5*np.dot(D,np.dot(Incov,D))
    return logL


for j in range(chain):
    X=np.empty([n,3])
    X[0,:] = [omega,h,-1.e100]
    X[0,2] = lnL(X[0,0], X[0,1])
    #  M-H
    for i in range(1,n):
        logL_initial = X[i-1,2]
        Omega_next = np.random.normal(X[i-1,0],0.01)
        h_next     = np.random.normal(X[i-1,1],0.01)   
        logL_next  = lnL(Omega_next,h_next)
        
        if logL_initial < logL_next: 
            X[i,0] = Omega_next
            X[i,1] = h_next
            X[i,2] = logL_next
        else:
            alpha = np.random.uniform()
   
            if(logL_next - logL_initial > np.log(alpha)):
                    X[i,0] = Omega_next
                    X[i,1] = h_next
                    X[i,2] = logL_next
            else:
                X[i,0] = X[i-1,0]
                X[i,1] = X[i-1,1]
                X[i,2] = logL_initial

omega_m_chain_mean=[]
omega_m_chain_var=[]

h_chain_mean=[]
h_chain_var=[]

omega_m_mean=np.mean(X[:,0])
h_mean=np.mean(X[:,1])
        
omega_m_var=np.var(X[:,0])
h_var=np.var(X[:,1])
        
omega_m_chain_mean.append(omega_m_mean)
h_chain_mean.append(h_mean)
        
omega_m_chain_var.append(omega_m_var)
h_chain_var.append(h_var)

In [9]:
omega_m_sigmasq_chain=np.mean(omega_m_chain_var)
omega_m_sigmasq_mean=np.var(omega_m_chain_mean)
Ratio_omega_m= ((chain - 1.0)*omega_m_sigmasq_chain + omega_m_sigmasq_mean)/(omega_m_sigmasq_chain*chain)

print("value of Gelman-Rubin Convergence ratio for Omega_m by taking 500 MCMC sample is "+str(Ratio_omega_m))

value of Gelman-Rubin Convergence ratio for Omega_m by taking 500 MCMC sample is 0.9600000000000001


In [10]:
h_sigmasq_chain=np.mean(h_chain_var)
h_sigmasq_mean=np.var(h_chain_mean)

Ratio_h= ((chain - 1.0)*h_sigmasq_chain + h_sigmasq_mean)/(h_sigmasq_chain*chain)

print("value of Gelman-Rubin Convergence ratio for h by taking  500 MCMC sample is "+str(Ratio_h))

value of Gelman-Rubin Convergence ratio for h by taking  500 MCMC sample is 0.96


**since convergence ratio is nearly equal to 1 so the chain converges**